In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
# required by pd.read_csv. It expects an object with a .read() method.
from datetime import datetime, timedelta
from glob import glob
from bs4 import BeautifulSoup
import gzip
import os 

In [3]:
def getOverlap(mesaid, overlap_file="./data/mesa/mesa-actigraphy-psg-overlap.csv"):
    
    df = pd.read_csv(overlap_file)
    df = df[df["mesaid"] == mesaid]
    if df.empty:
        print("ERROR: mesaid %d not found in overlap file" % (mesaid))
        return None
    return df.values[0][1] # (mesaid, line, timeactigraph, timepsg)

def extractAcigraphy(mesaid):
    # Start to extract Actigraphy actValue
    try:
        act_file = "data/mesa/actigraphy/mesa-sleep-%04d.csv.gz" % (mesaid)
        df = load_mesa(act_file)
        return df
    except:
        return None

def extractPSG(mesaid):
    # Start to extract PSG sleep scores
    profusion_file = "./data/mesa/polysomnography/annotations-events-profusion/mesa-sleep-%04d-profusion.xml.gz" % (mesaid)
    try:
        with gzip.open(profusion_file) as f:
            soup = BeautifulSoup(f, 'xml')

        stages = []
        for c in soup.CMPStudyConfig.SleepStages:
            stages.append(c.string)

        return stages
    except:
        return None

def load_mesa(filename):
    """
        Load all the mesa files
    """
    
    df = pd.read_csv(filename, index_col="line")
    
    ts = get_timestamp("1/1/1900", df.iloc[0]["linetime"])
    pts = pd.Timestamp(ts)
    df["linetime"] = pd.date_range(pts, periods=df.shape[0], freq='30S')
    
    return df

def get_timestamp(start_date, start_time):
    return datetime.strptime(start_date + " " + start_time, '%m/%d/%Y %H:%M:%S')

def get_time_interval(n):
    minutes = n / 2
    hours = minutes/ 60
    rest_minutes = minutes - (hours * 60)
    rest_seconds = "30" if n%2 == 1 else "00"
    return "%02d:%02d:%s" % (hours, rest_minutes, rest_seconds)

In [6]:
def process(mesaid, task):
    """
    - mesaid: Integer representing a valid mesaid.
    
    - task:
         * task1 (PSG time)
         * task2 (24 hours)
    """
    print("Processing %d..." % (mesaid))
    
    # Extract Actigraphy data
    df = extractAcigraphy(mesaid)
    if df is None:
        print ("Actigraphy not found for mesaid %d. Aborting..." % (mesaid))
        return None
    
    # Extract PSG data
    stages = extractPSG(mesaid)
    if stages is None:
        print ("PSG not found for mesaid %d. Aborting..." % (mesaid))
        return None
    
    stages = list(map(int, stages))

    # If recording for more than 16 hours, we do not use this mesaid
    if len(stages) > 1920:
        print ("PSG recording was longer than 16 hours for mesaid %d." % (mesaid))
        print ("Aborting...")
        return None
    
    # Extract overlap index
    overlapidx = getOverlap(mesaid)

    if overlapidx is None:
        print ("Problems with mesaid %d. Aborting..." % (mesaid))
        return None
    
    #return df, stages, overlapidx
    
    # Creates an extra col with PSG data and fills up data with stages
    df["stage"] = None
    df.loc[overlapidx:overlapidx+len(stages)-1, "stage"] = stages
    
    # Task 1: only PSG data is kept
    if task == 1:
        df = df[~df["stage"].isnull()]
        
    elif task == 2: # aims to get 960 (8 hours) intervals before and after PSG 
        startidx = max(0, overlapidx - 960)
        endidx = min(df.shape[0], overlapidx+len(stages)-1 + 960)
        df = df.loc[startidx: endidx]
    
    # Generates the ground truth data:
    if task == 1:
        df["gt"] = (df["stage"] > 0)
    
    elif task == 2:
        # ...uses GT as the PSG states and uses "interval" data for the rest....
        df["gt"] = None
        beforePSG = df.loc[:overlapidx-1].copy()
        duringPSG = df.loc[overlapidx: overlapidx+len(stages)-2].copy()
        afterPSG = df.loc[overlapidx+len(stages)-1:].copy()

        beforePSG["gt"] = beforePSG["interval"].apply(lambda x: x in ["REST", "REST-S"])
        afterPSG["gt"] = afterPSG["interval"].apply(lambda x: x in ["REST", "REST-S"])
        duringPSG["gt"] =  (duringPSG["stage"] > 0)
        df = pd.concat((beforePSG,duringPSG,afterPSG))
        
    df["gt"] = df["gt"].astype(np.int)
        
    dflenght = df.shape[0]
    print("Final df duration %s (%d intervals) -- From %s to %s" % (get_time_interval(dflenght),dflenght, df.head(1)["linetime"].values[0], df.tail(1)["linetime"].values[0]))

    # Resets index
    df = df.reset_index(drop=True)
    
    return df
    

In [7]:
 process(2896, 1)  # 2896, 4012 should be there...

Processing 2896...
Final df duration 11:00:30 (1439 intervals) -- From 1900-01-01T21:00:00.000000000 to 1900-01-02T08:59:00.000000000


,mesaid,linetime,offwrist,activity,marker,whitelight,redlight,greenlight,bluelight,wake,interval,dayofweek,daybymidnight,daybynoon,stage,gt
0,2896,1900-01-01 21:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXCLUDED,3,1,1,0,0
1,2896,1900-01-01 21:00:30,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXCLUDED,3,1,1,0,0
2,2896,1900-01-01 21:01:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXCLUDED,3,1,1,0,0
3,2896,1900-01-01 21:01:30,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXCLUDED,3,1,1,0,0
4,2896,1900-01-01 21:02:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXCLUDED,3,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,2896,1900-01-02 08:57:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXCLUDED,4,2,1,0,0
1435,2896,1900-01-02 08:57:30,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXCLUDED,4,2,1,0,0
1436,2896,1900-01-02 08:58:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXCLUDED,4,2,1,0,0
1437,2896,1900-01-02 08:58:30,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXCLUDED,4,2,1,0,0


In [10]:
TASK = 1
outdir = "datasets"

variables_file = "./data/mesa/mesa-sleep-dataset-0.3.0.csv"
variables = pd.read_csv(variables_file)
ids = list(variables["mesaid"].unique())

problem = []
okays = 0
empties = []
processed = 0

for mesaid in ids[:]:
    
    processed += 1
    
    print("*" * 80)    
    df = process(mesaid, task=TASK)
    print("*" * 80)
    
    if df is None:
        print("Could not get data for mesaid %d" % (mesaid))
        problem.append(mesaid)
        continue
    
    if df[(df["interval"] == "EXCLUDED")].shape[0] > 0:
        print("FOUND %d rows to be excluded" % (df[(df["interval"] == "EXCLUDED")].shape[0]))
    
    if (df[df["interval"] == "EXCLUDED"].shape[0] == df.shape[0]) or df.empty:
        print("ERROR: All intervals were excluded.")
        print("MesaID %d is empty..." % mesaid)
        empties.append(mesaid)
        continue
    
    #df = df[df["interval"] != "EXCLUDED"] # TODO: not sure if I should keep or remove the excluded rows.
   
    outfile = "mesa_%000d_task%s.csv" % (mesaid, TASK)
    outpath = os.path.join(outdir, outfile)
        
    df.to_csv(outpath, index=False)
    okays += 1
    
print("Done with %d okay out of %d possible ids" % (okays, processed))


********************************************************************************
Processing 1...
Final df duration 11:00:30 (1439 intervals) -- From 1900-01-01T20:30:00.000000000 to 1900-01-02T08:29:00.000000000
********************************************************************************
********************************************************************************
Processing 2...
Final df duration 10:00:30 (1319 intervals) -- From 1900-01-01T21:00:00.000000000 to 1900-01-02T07:59:00.000000000
********************************************************************************
********************************************************************************
Processing 6...
Final df duration 08:00:30 (1079 intervals) -- From 1900-01-01T22:00:00.000000000 to 1900-01-02T06:59:00.000000000
********************************************************************************
********************************************************************************
Processing 10...
Final df duration 09:00

ERROR: mesaid 85 not found in overlap file
Problems with mesaid 85. Aborting...
********************************************************************************
Could not get data for mesaid 85
********************************************************************************
Processing 87...
ERROR: mesaid 87 not found in overlap file
Problems with mesaid 87. Aborting...
********************************************************************************
Could not get data for mesaid 87
********************************************************************************
Processing 93...
PSG not found for mesaid 93. Aborting...
********************************************************************************
Could not get data for mesaid 93
********************************************************************************
Processing 99...
Final df duration 10:00:30 (1259 intervals) -- From 1900-01-01T19:30:00.000000000 to 1900-01-02T05:59:00.000000000
************************************************

Final df duration 08:00:30 (1079 intervals) -- From 1900-01-01T21:00:00.000000000 to 1900-01-02T05:59:00.000000000
********************************************************************************
********************************************************************************
Processing 168...
PSG not found for mesaid 168. Aborting...
********************************************************************************
Could not get data for mesaid 168
********************************************************************************
Processing 169...
Final df duration 08:00:30 (1079 intervals) -- From 1900-01-01T22:00:00.000000000 to 1900-01-02T06:59:00.000000000
********************************************************************************
********************************************************************************
Processing 171...
Final df duration 09:00:30 (1199 intervals) -- From 1900-01-01T23:00:00.000000000 to 1900-01-02T08:59:00.000000000
***************************************

Final df duration 09:00:30 (1199 intervals) -- From 1900-01-01T20:00:00.000000000 to 1900-01-02T05:59:00.000000000
********************************************************************************
********************************************************************************
Processing 275...
Final df duration 09:00:30 (1199 intervals) -- From 1900-01-01T22:30:00.000000000 to 1900-01-02T08:29:00.000000000
********************************************************************************
********************************************************************************
Processing 282...
Final df duration 09:00:30 (1199 intervals) -- From 1900-01-01T22:00:00.000000000 to 1900-01-02T07:59:00.000000000
********************************************************************************
********************************************************************************
Processing 284...
ERROR: mesaid 284 not found in overlap file
Problems with mesaid 284. Aborting...
*********************************

Final df duration 12:00:00 (1440 intervals) -- From 1900-01-01T21:00:00.000000000 to 1900-01-02T08:59:30.000000000
********************************************************************************
FOUND 22 rows to be excluded
********************************************************************************
Processing 347...
Final df duration 09:00:30 (1199 intervals) -- From 1900-01-01T23:00:00.000000000 to 1900-01-02T08:59:00.000000000
********************************************************************************
********************************************************************************
Processing 350...
Final df duration 11:00:30 (1439 intervals) -- From 1900-01-01T21:00:00.000000000 to 1900-01-02T08:59:00.000000000
********************************************************************************
********************************************************************************
Processing 359...
Final df duration 10:00:30 (1259 intervals) -- From 1900-01-01T21:30:00.000000000 to 1

...

Final df duration 13:00:30 (1619 intervals) -- From 1900-01-01T23:30:00.000000000 to 1900-01-02T12:59:00.000000000
********************************************************************************
FOUND 55 rows to be excluded
********************************************************************************
Processing 6383...
Final df duration 09:00:30 (1199 intervals) -- From 1900-01-01T22:00:00.000000000 to 1900-01-02T07:59:00.000000000
********************************************************************************
********************************************************************************
Processing 6384...
Final df duration 11:00:30 (1439 intervals) -- From 1900-01-01T22:00:00.000000000 to 1900-01-02T09:59:00.000000000
********************************************************************************
FOUND 25 rows to be excluded
********************************************************************************
Processing 6385...
ERROR: mesaid 6385 not found in overlap file
Problems 

********************************************************************************
Processing 6447...
Final df duration 09:00:30 (1199 intervals) -- From 1900-01-01T22:00:00.000000000 to 1900-01-02T07:59:00.000000000
********************************************************************************
********************************************************************************
Processing 6450...
Final df duration 10:00:00 (1256 intervals) -- From 1900-01-01T22:00:00.000000000 to 1900-01-02T08:27:30.000000000
********************************************************************************
********************************************************************************
Processing 6452...
Final df duration 08:00:30 (1079 intervals) -- From 1900-01-01T22:00:00.000000000 to 1900-01-02T06:59:00.000000000
********************************************************************************
********************************************************************************
Processing 6454...
Final df dur

ERROR: mesaid 6528 not found in overlap file
Problems with mesaid 6528. Aborting...
********************************************************************************
Could not get data for mesaid 6528
********************************************************************************
Processing 6538...
Final df duration 11:00:30 (1439 intervals) -- From 1900-01-01T23:00:00.000000000 to 1900-01-02T10:59:00.000000000
********************************************************************************
********************************************************************************
Processing 6549...
ERROR: mesaid 6549 not found in overlap file
Problems with mesaid 6549. Aborting...
********************************************************************************
Could not get data for mesaid 6549
********************************************************************************
Processing 6550...
PSG not found for mesaid 6550. Aborting...
*************************************************************

Final df duration 09:00:30 (1199 intervals) -- From 1900-01-01T19:00:00.000000000 to 1900-01-02T04:59:00.000000000
********************************************************************************
********************************************************************************
Processing 6653...
Final df duration 11:00:30 (1439 intervals) -- From 1900-01-01T21:00:00.000000000 to 1900-01-02T08:59:00.000000000
********************************************************************************
FOUND 52 rows to be excluded
********************************************************************************
Processing 6655...
Final df duration 08:00:30 (1079 intervals) -- From 1900-01-01T23:00:00.000000000 to 1900-01-02T07:59:00.000000000
********************************************************************************
********************************************************************************
Processing 6656...
Final df duration 11:00:30 (1439 intervals) -- From 1900-01-01T21:00:00.000000000 t

Processing 6751...
Final df duration 11:00:00 (1436 intervals) -- From 1900-01-01T19:02:00.000000000 to 1900-01-02T06:59:30.000000000
********************************************************************************
********************************************************************************
Processing 6754...
Final df duration 10:00:00 (1200 intervals) -- From 1900-01-01T22:00:00.000000000 to 1900-01-02T07:59:30.000000000
********************************************************************************
********************************************************************************
Processing 6755...
Final df duration 11:00:30 (1439 intervals) -- From 1900-01-01T22:00:00.000000000 to 1900-01-02T09:59:00.000000000
********************************************************************************
FOUND 23 rows to be excluded
********************************************************************************
Processing 6756...
Final df duration 10:00:30 (1319 intervals) -- From 1900-01-01T2